object detection only

In [1]:
import cv2
from ultralytics import YOLO
import numpy as np

# Load your trained model
model = YOLO(r'E:\elevatetrsest\CamEDGE counting\bestv2.pt')

def detect_warehouse_video(video_path, output_path=None, conf_threshold=0.5):
    """
    Detect objects in warehouse video and count them
    
    Args:
        video_path: Path to input video
        output_path: Path to save output video (optional)
        conf_threshold: Confidence threshold for detections
    """
    
    # Open video
    cap = cv2.VideoCapture(video_path)
    
    if output_path:
        # Get video properties
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        
        # Create video writer
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    frame_count = 0
    total_detections = {}
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
            
        frame_count += 1
        
        # Run detection
        results = model(frame, conf=conf_threshold)
        
        # Process results
        frame_detections = {}
        for result in results:
            if result.boxes is not None:
                for box in result.boxes:
                    class_id = int(box.cls)
                    class_name = model.names[class_id]
                    confidence = float(box.conf)
                    
                    # Count detections
                    frame_detections[class_name] = frame_detections.get(class_name, 0) + 1
                    total_detections[class_name] = total_detections.get(class_name, 0) + 1
        
        # Draw results on frame
        annotated_frame = results[0].plot()
        
        # Add count text
        y_offset = 30
        for class_name, count in frame_detections.items():
            cv2.putText(annotated_frame, f"{class_name}: {count}", 
                       (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            y_offset += 25
        
        # Save frame if output specified
        if output_path:
            out.write(annotated_frame)
        
        # Display frame (optional - comment out for headless)
        cv2.imshow('Warehouse Detection', annotated_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
        # Print progress every 100 frames
        if frame_count % 100 == 0:
            print(f"Processed {frame_count} frames - Current detections: {frame_detections}")
    
    # Cleanup
    cap.release()
    if output_path:
        out.release()
    cv2.destroyAllWindows()
    
    print(f"\n🎉 Video processing complete!")
    print(f"📝 Total frames processed: {frame_count}")
    print(f"📊 Total object detections: {total_detections}")
    
    return total_detections

# Usage examples:
print("🚀 Warehouse Video Detection System Ready!")
print("\n📝 Usage:")
print("1. Process video and display: detect_warehouse_video('warehouse_video.mp4')")
print("2. Process and save output: detect_warehouse_video('input.mp4', 'output.mp4')")
print("3. With custom confidence: detect_warehouse_video('video.mp4', conf_threshold=0.6)")

# Example usage:
# detections = detect_warehouse_video('your_warehouse_video.mp4', 'output_with_detections.mp4')

🚀 Warehouse Video Detection System Ready!

📝 Usage:
1. Process video and display: detect_warehouse_video('warehouse_video.mp4')
2. Process and save output: detect_warehouse_video('input.mp4', 'output.mp4')
3. With custom confidence: detect_warehouse_video('video.mp4', conf_threshold=0.6)


In [14]:
# Process warehouse video
detections = detect_warehouse_video(r'E:\elevatetrsest\CamEDGE counting\test12_warehouse.mp4', 'output12.mp4')

0: 384x640 4 boxs, 2 persons, 309.3ms
Speed: 4.4ms preprocess, 309.3ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 boxs, 1 person, 37.8ms
Speed: 1.1ms preprocess, 37.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 box, 1 person, 38.0ms
Speed: 1.1ms preprocess, 38.0ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 box, 1 person, 35.3ms
Speed: 1.7ms preprocess, 35.3ms inference, 3.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 35.5ms
Speed: 1.0ms preprocess, 35.5ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 35.3ms
Speed: 1.3ms preprocess, 35.3ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 35.3ms
Speed: 2.0ms preprocess, 35.3ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 35.6ms
Speed: 1.3ms preprocess, 35.6ms inference, 2.2ms postproc

object detection + tracking

In [2]:
import cv2
from ultralytics import YOLO
import numpy as np

# Load your trained model
model = YOLO(r'E:\elevatetrsest\CamEDGE counting\bestv2.pt')

# Define the classes your model was trained on (for clarity, though model.names should have this)
CLASSES = ['box', 'box_broken', 'forklift', 'open_package', 'package', 'pallets', 'person']

def detect_and_track_warehouse_video(video_path, output_path=None, conf_threshold=0.5, tracker_config="bytetrack.yaml"):
    """
    Detect and track objects in warehouse video and count them per frame.

    Args:
        video_path: Path to input video
        output_path: Path to save output video (optional)
        conf_threshold: Confidence threshold for detections
        tracker_config: Path to the tracker configuration file (e.g., "bytetrack.yaml" or "botsort.yaml")
    """

    # Open video
    cap = cv2.VideoCapture(video_path)

    writer = None
    if output_path:
        # Get video properties
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        # Create video writer
        fourcc = cv2.VideoWriter_fourcc(*'mp4v') # or 'XVID'
        writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_count = 0
    total_detections_summary = {} # To store cumulative count of unique items seen based on first appearance

    # For tracking, unique IDs seen in the video
    # This simple approach counts an object ID once when it first appears.
    # For more sophisticated counting (e.g., current count on screen),
    # you'd need to manage active track IDs per frame.
    tracked_ids_ever_seen = set()

    while True:
        ret, frame = cap.read()
        if not ret:
            print("End of video or error reading frame.")
            break

        frame_count += 1

        # Run detection AND tracking
        # Use persist=True to keep track IDs consistent across frames
        # The tracker can be "bytetrack.yaml" or "botsort.yaml"
        results = model.track(frame, conf=conf_threshold, persist=True, tracker=tracker_config)

        # Process results
        current_frame_detections_count = {class_name: 0 for class_name in CLASSES} # Counts for current frame
        annotated_frame = frame.copy() # Make a copy for annotation if using results[0].plot() directly is not enough

        if results[0].boxes is not None and results[0].boxes.id is not None:
            # Get bounding boxes, confidences, class IDs, and track IDs
            boxes = results[0].boxes.xyxy.cpu().numpy() # Bounding boxes (x1, y1, x2, y2)
            confs = results[0].boxes.conf.cpu().numpy()  # Confidences
            class_ids = results[0].boxes.cls.cpu().numpy().astype(int) # Class IDs
            track_ids = results[0].boxes.id.cpu().numpy().astype(int) # Track IDs

            for i in range(len(boxes)):
                x1, y1, x2, y2 = map(int, boxes[i])
                conf = confs[i]
                class_id = class_ids[i]
                track_id = track_ids[i]

                class_name = model.names[class_id]

                # Count for current frame
                current_frame_detections_count[class_name] += 1

                # Update total unique detections summary
                if track_id not in tracked_ids_ever_seen:
                    total_detections_summary[class_name] = total_detections_summary.get(class_name, 0) + 1
                    tracked_ids_ever_seen.add(track_id)

                # Annotate the frame with bounding box, class name, confidence, and track ID
                label = f"ID:{track_id} {class_name} {conf:.2f}"
                color = (0, 255, 0) # Green for bounding box
                if class_name == "box_broken":
                    color = (0,0,255) # Red for broken box
                elif class_name == "open_package":
                    color = (0,165,255) # Orange for open package

                cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(annotated_frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

        else: # No detections or no track IDs in this frame
            # Use the original frame plot if you want to see "no detections" or default YOLO annotations
            # annotated_frame = results[0].plot()
            # For this manual drawing, if no detections, annotated_frame remains the current frame
            pass


        # Add current frame count text
        y_offset = 30
        cv2.putText(annotated_frame, f"Frame: {frame_count}",
                    (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
        y_offset += 25
        for class_name_txt, count_txt in current_frame_detections_count.items():
            if count_txt > 0: # Only display if count > 0 for cleaner UI
                cv2.putText(annotated_frame, f"{class_name_txt}: {count_txt}",
                            (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
                y_offset += 25

        # Save frame if output specified
        if writer:
            writer.write(annotated_frame)

        # Display frame (optional - comment out for headless)
        cv2.imshow('Warehouse Detection and Tracking', annotated_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        # Print progress every 100 frames
        if frame_count % 100 == 0:
            print(f"Processed {frame_count} frames. Current frame detections: {current_frame_detections_count}")

    # Cleanup
    cap.release()
    if writer:
        writer.release()
    cv2.destroyAllWindows()

    print(f"\n🎉 Video processing complete!")
    print(f"📝 Total frames processed: {frame_count}")
    print(f"📊 Summary of unique tracked objects detected: {total_detections_summary}")

    return total_detections_summary

# Usage examples:
print("🚀 Warehouse Video Detection and Tracking System Ready!")
print("\n📝 Usage:")
print("1. Process video and display: detect_and_track_warehouse_video('your_video.mp4')")
print("2. Process and save output: detect_and_track_warehouse_video('input.mp4', 'output_tracked.mp4')")
print("3. With custom confidence: detect_and_track_warehouse_video('video.mp4', conf_threshold=0.6)")
print("4. With different tracker (e.g., BoT-SORT): detect_and_track_warehouse_video('video.mp4', tracker_config='botsort.yaml')")

# Example usage (uncomment and replace 'your_warehouse_video.mp4'):
# detections_summary = detect_and_track_warehouse_video(
# r'E:\your_path_to_video\your_warehouse_video.mp4',
# r'E:\your_path_to_video\output_with_tracking.mp4',
#     conf_threshold=0.5,
#     tracker_config="bytetrack.yaml" # or "botsort.yaml"
# )
# print(detections_summary)

WARNING Ultralytics settings reset to default values. This may be due to a possible problem with your settings or a recent ultralytics package update. 
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\Shiva\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
🚀 Warehouse Video Detection and Tracking System Ready!

📝 Usage:
1. Process video and display: detect_and_track_warehouse_video('your_video.mp4')
2. Process and save output: detect_and_track_warehouse_video('input.mp4', 'output_tracked.mp4')
3. With custom confidence: detect_and_track_warehouse_video('video.mp4', conf_threshold=0.6)
4. With different tracker (e.g., BoT-SORT): detect_and_track_warehouse_video('video.mp4', tracker_config='botsort.yaml')


In [3]:
# Example usage (uncomment and replace 'your_warehouse_video.mp4'):
detections_summary = detect_and_track_warehouse_video(
r'E:\elevatetrsest\CamEDGE counting\test\test3_warehouse.mp4',
r'E:\elevatetrsest\CamEDGE counting\tracking_output\output3.mp4',
    conf_threshold=0.5,
    tracker_config="bytetrack.yaml" # or "botsort.yaml"
)
print(detections_summary)

requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...
   ---------------------------------------- 1.5/1.5 MB 5.2 MB/s eta 0:00:00

requirements: AutoUpdate success  5.9s
WARNING requirements: Restart runtime or rerun command for updates to take effect


0: 352x640 3 boxs, 131.0ms
Speed: 8.1ms preprocess, 131.0ms inference, 233.9ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 3 boxs, 105.6ms
Speed: 3.1ms preprocess, 105.6ms inference, 5.3ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 3 boxs, 106.0ms
Speed: 4.2ms preprocess, 106.0ms inference, 4.7ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 3 boxs, 85.1ms
Speed: 2.5ms preprocess, 85.1ms inference, 3.3ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 3 boxs, 77.4ms
Speed: 3.0ms preprocess, 77.4ms inference, 5.3ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 3 boxs, 72.7ms
Speed: 4.1ms preprocess, 72.7ms inference, 2.2ms postpro

object tracking (only box)

In [1]:
import cv2
from ultralytics import YOLO
import numpy as np

# Load your trained model
model = YOLO(r'E:\elevatetrsest\CamEDGE counting\bestv2.pt') # Use your model path

# Define the classes your model was trained on
CLASSES = ['box', 'box_broken', 'forklift', 'open_package', 'package', 'pallets', 'person']
CLASS_TO_TRACK_EXCLUSIVELY = 'box'

def detect_and_selectively_track_video(video_path, output_path=None, conf_threshold=0.5, tracker_config="bytetrack.yaml"):
    """
    Detect all objects, but only display track IDs for the 'box' class.
    Counts all objects per frame and unique 'box' instances.

    Args:
        video_path: Path to input video
        output_path: Path to save output video (optional)
        conf_threshold: Confidence threshold for detections
        tracker_config: Path to the tracker configuration file
    """
    cap = cv2.VideoCapture(video_path)
    writer = None

    if output_path:
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fourcc = cv2.VideoWriter_fourcc(*'mp4v') # or 'XVID'
        writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_count = 0
    
    # For counting unique 'box' objects based on their track ID
    total_unique_tracked_boxes = 0
    seen_box_track_ids = set()

    # For counting all object types per frame
    current_frame_object_counts = {class_name: 0 for class_name in CLASSES}
    # For overall summary of any object detected at least once (can be based on track ID if available)
    # This can be adjusted if you only want summary for boxes.
    # For now, it will count any object with a track ID once.
    overall_objects_summary = {class_name: 0 for class_name in CLASSES}
    overall_seen_track_ids = set()


    while True:
        ret, frame = cap.read()
        if not ret:
            print("End of video or error reading frame.")
            break
        frame_count += 1
        annotated_frame = frame.copy()

        # Run detection AND tracking
        # persist=True is important for consistent track IDs
        results = model.track(frame, conf=conf_threshold, persist=True, tracker=tracker_config, verbose=False)

        # Reset per-frame counts
        current_frame_object_counts = {class_name: 0 for class_name in CLASSES}

        if results[0].boxes is not None: # Check if there are any detections
            boxes_data = results[0].boxes.xyxy.cpu().numpy()
            confs_data = results[0].boxes.conf.cpu().numpy()
            class_ids_data = results[0].boxes.cls.cpu().numpy().astype(int)
            
            # track_ids might be None if no objects are being tracked yet or tracker failed
            track_ids_data = None
            if results[0].boxes.id is not None:
                track_ids_data = results[0].boxes.id.cpu().numpy().astype(int)

            for i in range(len(boxes_data)):
                x1, y1, x2, y2 = map(int, boxes_data[i])
                conf = confs_data[i]
                class_id = class_ids_data[i]
                class_name = model.names[class_id]

                # Increment count for the current frame
                current_frame_object_counts[class_name] += 1
                
                label = f"{class_name} {conf:.2f}"
                track_id_for_this_object = None

                if track_ids_data is not None: # Ensure track IDs exist
                    track_id_for_this_object = track_ids_data[i]
                    
                    # Add track ID to label ONLY if it's the CLASS_TO_TRACK_EXCLUSIVELY
                    if class_name == CLASS_TO_TRACK_EXCLUSIVELY:
                        label = f"ID:{track_id_for_this_object} {class_name} {conf:.2f}"
                        if track_id_for_this_object not in seen_box_track_ids:
                            seen_box_track_ids.add(track_id_for_this_object)
                            total_unique_tracked_boxes += 1
                    
                    # For overall summary, count any tracked object once
                    if track_id_for_this_object not in overall_seen_track_ids:
                        overall_objects_summary[class_name] = overall_objects_summary.get(class_name, 0) + 1
                        overall_seen_track_ids.add(track_id_for_this_object)


                # --- Visuals: Bounding box for all ---
                box_color = (0, 255, 0) # Default Green
                if class_name == "box_broken":
                    box_color = (0,0,255) # Red
                elif class_name == "open_package":
                    box_color = (0,165,255) # Orange
                elif class_name == "forklift":
                    box_color = (255,0,0) # Blue
                elif class_name == "pallets":
                    box_color = (255,255,0) # Yellow
                elif class_name == "person":
                    box_color = (128,0,128) # Purple
                elif class_name == CLASS_TO_TRACK_EXCLUSIVELY: # Ensure tracked box has a distinct color if not already set
                     box_color = (0, 255, 0) # Green for tracked boxes

                cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), box_color, 2)
                cv2.putText(annotated_frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, box_color, 2)
        
        # Display current frame object counts
        y_offset = 30
        cv2.putText(annotated_frame, f"Frame: {frame_count}",
                    (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2) # White text
        y_offset += 25
        for cls_name_txt, count_txt in current_frame_object_counts.items():
            if count_txt > 0: # Only display if count > 0
                cv2.putText(annotated_frame, f"{cls_name_txt}: {count_txt}",
                            (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2) # Yellow text
                y_offset += 25

        if writer:
            writer.write(annotated_frame)

        cv2.imshow(f'Warehouse Tracking (Tracking IDs for "{CLASS_TO_TRACK_EXCLUSIVELY}" only)', annotated_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        if frame_count % 100 == 0:
            print(f"Processed {frame_count} frames. Unique '{CLASS_TO_TRACK_EXCLUSIVELY}' count: {total_unique_tracked_boxes}. Current frame counts: {current_frame_object_counts}")

    cap.release()
    if writer:
        writer.release()
    cv2.destroyAllWindows()

    print(f"\n🎉 Video processing complete!")
    print(f"📝 Total frames processed: {frame_count}")
    print(f"📦 Total unique '{CLASS_TO_TRACK_EXCLUSIVELY}' objects tracked: {total_unique_tracked_boxes}")
    print(f"📊 Overall summary of unique objects (any class with track ID) detected: {overall_objects_summary}")
    
    return total_unique_tracked_boxes, overall_objects_summary




In [2]:
# --- Usage Example ---
if __name__ == "__main__":
    print(f"🚀 Warehouse Video Detection (Selective Tracking for '{CLASS_TO_TRACK_EXCLUSIVELY}') System Ready!")
    print("\n📝 Usage:")
    print("1. Process video and display: detect_and_selectively_track_video('your_video.mp4')")
    print("2. Process and save output: detect_and_selectively_track_video('input.mp4', 'output_tracked.mp4')")

    # --- CONFIGURE YOUR VIDEO PATH HERE ---
    # video_input_path = r'E:\your_path_to_video\your_warehouse_video.mp4' # Example Windows path
    video_input_path = r'E:\elevatetrsest\CamEDGE counting\test\test3_warehouse.mp4' # REPLACE THIS
    video_output_path = r'E:\elevatetrsest\CamEDGE counting\tracking_output\output3.mp4' # REPLACE THIS (OPTIONAL)

    if video_input_path == r'path_to_your_warehouse_video.mp4': # Check if placeholder is still there
        print("\n⚠️ PLEASE UPDATE 'video_input_path' in the script with your actual video file path.")
    else:
        try:
            print(f"\nProcessing video: {video_input_path}")
            # Example: Process and display only
            unique_boxes, overall_summary = detect_and_selectively_track_video(video_input_path, conf_threshold=0.4)
            
            # Example: Process and save
            # unique_boxes, overall_summary = detect_and_selectively_track_video(video_input_path, video_output_path, conf_threshold=0.4)
            
            print(f"\nFinal count of unique '{CLASS_TO_TRACK_EXCLUSIVELY}' objects: {unique_boxes}")
            # print(f"Final overall summary of unique objects: {overall_summary}")

        except FileNotFoundError:
            print(f"ERROR: Video file not found at '{video_input_path}'. Please check the path.")
        except Exception as e:
            print(f"An error occurred: {e}")
            print("Ensure your video path is correct and OpenCV and Ultralytics are installed properly.")

🚀 Warehouse Video Detection (Selective Tracking for 'box') System Ready!

📝 Usage:
1. Process video and display: detect_and_selectively_track_video('your_video.mp4')
2. Process and save output: detect_and_selectively_track_video('input.mp4', 'output_tracked.mp4')

Processing video: E:\elevatetrsest\CamEDGE counting\test\test3_warehouse.mp4
Processed 100 frames. Unique 'box' count: 5. Current frame counts: {'box': 2, 'box_broken': 0, 'forklift': 1, 'open_package': 0, 'package': 0, 'pallets': 0, 'person': 0}
Processed 200 frames. Unique 'box' count: 7. Current frame counts: {'box': 1, 'box_broken': 0, 'forklift': 0, 'open_package': 0, 'package': 0, 'pallets': 0, 'person': 0}
Processed 300 frames. Unique 'box' count: 7. Current frame counts: {'box': 0, 'box_broken': 0, 'forklift': 0, 'open_package': 0, 'package': 0, 'pallets': 0, 'person': 1}
Processed 400 frames. Unique 'box' count: 14. Current frame counts: {'box': 1, 'box_broken': 0, 'forklift': 1, 'open_package': 0, 'package': 0, 'pal

object tracking (only box) + centroid and tail

In [ ]:
# import cv2
# from ultralytics import YOLO
# import numpy as np
# from collections import deque

# # Load your trained model
# model = YOLO(r'E:\elevatetrsest\CamEDGE counting\bestv2.pt') # Use your model path

# # Define the classes your model was trained on
# CLASSES = ['box', 'box_broken', 'forklift', 'open_package', 'package', 'pallets', 'person']
# TARGET_TRACK_CLASS = 'box' # Class to apply special tracking visuals

# # Store history of centroids for tail
# # Format: {track_id: deque([(x1,y1), (x2,y2), ...])}
# track_history = {}
# TAIL_LENGTH = 30  # Number of frames for the tail

# def detect_and_track_warehouse_video(video_path, output_path=None, conf_threshold=0.5, tracker_config="bytetrack.yaml"):
#     """
#     Detect objects, specifically track "box" class with centroid and tail.
#     Counts other objects per frame as well.

#     Args:
#         video_path: Path to input video
#         output_path: Path to save output video (optional)
#         conf_threshold: Confidence threshold for detections
#         tracker_config: Path to the tracker configuration file
#     """
#     global track_history # Use the global track_history

#     cap = cv2.VideoCapture(video_path)
#     writer = None

#     if output_path:
#         fps = int(cap.get(cv2.CAP_PROP_FPS))
#         width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
#         height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
#         fourcc = cv2.VideoWriter_fourcc(*'mp4v')
#         writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

#     frame_count = 0
#     # Summary for all unique items based on first appearance (can be adapted if needed)
#     total_detections_summary = {class_name: 0 for class_name in CLASSES}
#     tracked_ids_ever_seen = set()

#     while True:
#         ret, frame = cap.read()
#         if not ret:
#             print("End of video or error reading frame.")
#             break
#         frame_count += 1
#         annotated_frame = frame.copy()

#         # Run detection AND tracking
#         results = model.track(frame, conf=conf_threshold, persist=True, tracker=tracker_config, verbose=False)

#         current_frame_detections_count = {class_name: 0 for class_name in CLASSES}

#         if results[0].boxes is not None and results[0].boxes.id is not None:
#             boxes_data = results[0].boxes.xyxy.cpu().numpy()
#             confs_data = results[0].boxes.conf.cpu().numpy()
#             class_ids_data = results[0].boxes.cls.cpu().numpy().astype(int)
#             track_ids_data = results[0].boxes.id.cpu().numpy().astype(int)

#             for i in range(len(boxes_data)):
#                 x1, y1, x2, y2 = map(int, boxes_data[i])
#                 conf = confs_data[i]
#                 class_id = class_ids_data[i]
#                 track_id = track_ids_data[i]
#                 class_name = model.names[class_id]

#                 # Count all detected objects for the current frame
#                 current_frame_detections_count[class_name] += 1

#                 # Update total unique detections summary for all classes
#                 # This counts any object with a track ID once
#                 if track_id not in tracked_ids_ever_seen:
#                     total_detections_summary[class_name] = total_detections_summary.get(class_name, 0) + 1
#                     tracked_ids_ever_seen.add(track_id)

#                 # --- Visuals: Bounding box for all, special tracking for TARGET_TRACK_CLASS ---
#                 box_color = (0, 255, 0) # Default Green
#                 if class_name == "box_broken":
#                     box_color = (0,0,255) # Red
#                 elif class_name == "open_package":
#                     box_color = (0,165,255) # Orange
#                 elif class_name == "forklift":
#                     box_color = (255,0,0) # Blue
#                 elif class_name == "pallets":
#                     box_color = (255,255,0) # Yellow
#                 elif class_name == "person":
#                     box_color = (128,0,128) # Purple


#                 if class_name == TARGET_TRACK_CLASS:
#                     # Calculate centroid
#                     cx = (x1 + x2) // 2
#                     cy = (y1 + y2) // 2

#                     # Update track history
#                     if track_id not in track_history:
#                         track_history[track_id] = deque(maxlen=TAIL_LENGTH)
#                     track_history[track_id].append((cx, cy))

#                     # Draw centroid
#                     cv2.circle(annotated_frame, (cx, cy), 5, box_color, -1) # Solid circle

#                     # Draw tail
#                     for j in range(1, len(track_history[track_id])):
#                         if track_history[track_id][j - 1] is None or track_history[track_id][j] is None:
#                             continue
#                         thickness = int(np.sqrt(TAIL_LENGTH / float(j + 1)) * 2.0) # Tail gets thinner
#                         cv2.line(annotated_frame, track_history[track_id][j - 1], track_history[track_id][j], box_color, thickness)
                    
#                     # Label for the tracked "box" with ID
#                     label = f"ID:{track_id} {class_name}"
#                     cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), box_color, 2)
#                     cv2.putText(annotated_frame, label, (x1, y1 - 10),
#                                 cv2.FONT_HERSHEY_SIMPLEX, 0.6, box_color, 2)
#                 else:
#                     # Draw standard bounding box and label for other classes (no ID or tail)
#                     label = f"{class_name} {conf:.2f}"
#                     cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), box_color, 2)
#                     cv2.putText(annotated_frame, label, (x1, y1 - 10),
#                                 cv2.FONT_HERSHEY_SIMPLEX, 0.6, box_color, 2)
#         else:
#             # No detections or no track IDs in this frame
#             pass # annotated_frame remains the current frame


#         # Display current frame counts
#         y_offset = 30
#         cv2.putText(annotated_frame, f"Frame: {frame_count}",
#                     (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
#         y_offset += 25
#         for cls_name_txt, count_txt in current_frame_detections_count.items():
#             if count_txt > 0:
#                 cv2.putText(annotated_frame, f"{cls_name_txt}: {count_txt}",
#                             (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
#                 y_offset += 25

#         if writer:
#             writer.write(annotated_frame)

#         cv2.imshow('Warehouse Tracking - "box" focus', annotated_frame)
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break

#         if frame_count % 100 == 0:
#             active_box_tracks = sum(1 for tid in track_history if model.names[results[0].boxes.cls[np.where(results[0].boxes.id.cpu().numpy() == tid)[0][0]].cpu().numpy().astype(int)] == TARGET_TRACK_CLASS and len(track_history[tid]) > 0)
#             print(f"Processed {frame_count} frames. Current box tracks: {active_box_tracks}. Current frame counts: {current_frame_detections_count}")


#     cap.release()
#     if writer:
#         writer.release()
#     cv2.destroyAllWindows()
#     track_history.clear() # Clear history for next run if function is called again

#     print(f"\n🎉 Video processing complete!")
#     print(f"📝 Total frames processed: {frame_count}")
#     print(f"📊 Summary of unique tracked objects first seen: {total_detections_summary}")

#     return total_detections_summary



In [ ]:
# # --- Usage Example ---
# if __name__ == "__main__":
#     print("🚀 Warehouse Video Detection and 'box' Tracking System Ready!")
#     print("\n📝 Usage:")
#     print("1. Process video and display: detect_and_track_warehouse_video('your_video.mp4')")
#     print("2. Process and save output: detect_and_track_warehouse_video('input.mp4', 'output_tracked.mp4')")

#     # --- CONFIGURE YOUR VIDEO PATH HERE ---
#     # video_input_path = r'E:\your_path_to_video\your_warehouse_video.mp4'
#     video_input_path = r'E:\elevatetrsest\CamEDGE counting\test\test3_warehouse.mp4' # REPLACE THIS
#     video_output_path = r'E:\elevatetrsest\CamEDGE counting\tracking_output\output3.mp4' # REPLACE THIS (OPTIONAL)

#     if video_input_path == r'path_to_your_warehouse_video.mp4':
#         print("\n⚠️ PLEASE UPDATE 'video_input_path' in the script with your actual video file path.")
#     else:
#         try:
#             print(f"\nProcessing video: {video_input_path}")
#             # Example: Process and save
#             # detections_summary = detect_and_track_warehouse_video(video_input_path, video_output_path, conf_threshold=0.4)
            
#             # Example: Process and display only
#             detections_summary = detect_and_track_warehouse_video(video_input_path, conf_threshold=0.4)
#             print("\nFinal Summary of Unique Objects Detected:")
#             print(detections_summary)
#         except Exception as e:
#             print(f"An error occurred: {e}")
#             print("Ensure your video path is correct and OpenCV and Ultralytics are installed properly.")

🚀 Warehouse Video Detection and 'box' Tracking System Ready!

📝 Usage:
1. Process video and display: detect_and_track_warehouse_video('your_video.mp4')
2. Process and save output: detect_and_track_warehouse_video('input.mp4', 'output_tracked.mp4')

Processing video: E:\elevatetrsest\CamEDGE counting\test\test3_warehouse.mp4
An error occurred: index 0 is out of bounds for axis 0 with size 0
Ensure your video path is correct and OpenCV and Ultralytics are installed properly.


: 